In [3]:
import os,sys,subprocess,glob,cftime,importlib,pickle
from datetime import datetime
import xarray as xr
import numpy as np
import pandas as pd

def import_from(module, name):
    module = __import__(module, fromlist=[name])
    return getattr(module, name)

import warnings
warnings.filterwarnings("ignore")

sys.path.append('../')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from experiment_configuration.experiment import experiment
experiments = {}
for experiment_identifier in [f"c{i}" for i in range(1,6)] + [f"p{i}" for i in range(1,6)]:
    experiments[experiment_identifier] = experiment(import_from(f'experiment_configuration.{experiment_identifier}', 'config'))

In [5]:
all_variables = []
for name,exp in experiments.items():
    print('-'*20)
    print(name)
    run_dir = f"{exp.dir_archive}/GKLT/{exp.experiment_name}/{name}_000"
    file_dict = {}
    for fl in sorted(glob.glob(f"{run_dir}/*/*/*.nc")):

        if 'post' not in fl:
            realm = fl.split('/hist')[0].split('/')[-1]
            h_identifier = fl.split('.')[-3] 
            l = []
            nc = xr.open_dataset(fl)
            for var,e in nc.items():
                if e.dims == ('time', 'lat', 'lon'):
                    if e.time.shape[0] == 5:
                        l.append(var)
                        all_variables.append(var)

            if len(l) > 0:
                print(realm, h_identifier, l)
                file_dict[f"{realm} {h_identifier}"] = l
            #print(fl)
    print(file_dict)


--------------------
c1
atm h1 ['ICEFRAC', 'PRECC', 'PRECL', 'PS', 'PSL', 'QREFHT', 'RHREFHT', 'SST', 'TREFHT', 'TREFHTMN', 'TREFHTMX', 'Z500']
lnd h7 ['FIRE', 'GPP', 'HR', 'NBP', 'TLAI', 'TOTFIRE', 'TOTLITC', 'TOTVEGC', 'TWS']
{'atm h1': ['ICEFRAC', 'PRECC', 'PRECL', 'PS', 'PSL', 'QREFHT', 'RHREFHT', 'SST', 'TREFHT', 'TREFHTMN', 'TREFHTMX', 'Z500'], 'lnd h7': ['FIRE', 'GPP', 'HR', 'NBP', 'TLAI', 'TOTFIRE', 'TOTLITC', 'TOTVEGC', 'TWS']}
--------------------
p1
atm h1 ['ICEFRAC', 'PRECC', 'PRECL', 'PS', 'PSL', 'QREFHT', 'RHREFHT', 'SST', 'TREFHT', 'TREFHTMN', 'TREFHTMX', 'Z500']
lnd h7 ['FIRE', 'GPP', 'HR', 'NBP', 'TLAI', 'TOTFIRE', 'TOTLITC', 'TOTVEGC', 'TWS']
{'atm h1': ['ICEFRAC', 'PRECC', 'PRECL', 'PS', 'PSL', 'QREFHT', 'RHREFHT', 'SST', 'TREFHT', 'TREFHTMN', 'TREFHTMX', 'Z500'], 'lnd h7': ['FIRE', 'GPP', 'HR', 'NBP', 'TLAI', 'TOTFIRE', 'TOTLITC', 'TOTVEGC', 'TWS']}
--------------------
c2
atm h1 ['ICEFRAC', 'PRECC', 'PRECL', 'PS', 'PSL', 'QREFHT', 'RHREFHT', 'SST', 'TREFHT', 'TREFH

In [6]:
np.unique(all_variables)

array(['EFLX_GNET', 'EFLX_LH_TOT', 'FAREA_BURNED', 'FIRE', 'FLDS', 'FLNS',
       'FSDS', 'FSH', 'FSNO', 'FSNS', 'GPP', 'HR', 'ICEFRAC', 'LHFLX',
       'NBP', 'PBOT', 'PRECC', 'PRECL', 'PS', 'PSL', 'QFLX_EVAP_TOT',
       'QREFHT', 'QRUNOFF', 'QVEGT', 'RH2M', 'RH2M_U', 'RHREFHT', 'SHFLX',
       'SOILWATER_10CM', 'SST', 'TLAI', 'TOTFIRE', 'TOTLITC',
       'TOTSOILLIQ', 'TOTVEGC', 'TREFHT', 'TREFHTMN', 'TREFHTMX',
       'TSOI_10CM', 'TWS', 'U10', 'U200', 'U500', 'V200', 'V500', 'Z200',
       'Z500'], dtype='<U14')

In [16]:
available = pd.DataFrame(columns=list(experiments.keys()))
for var in np.unique(all_variables):
    available.loc[var,:] = ''
    for name,exp in experiments.items():
        run_dir = f"{exp.dir_archive}/GKLT/{exp.experiment_name}/{name}_000"
        file_dict = {}
        for fl in sorted(glob.glob(f"{run_dir}/*/*/*.nc")):

            if 'post' not in fl:
                realm = fl.split('/hist')[0].split('/')[-1]
                h_identifier = fl.split('.')[-3] 
                nc = xr.open_dataset(fl)
                for vv,e in nc.items():
                    if vv == var:
                        if e.dims == ('time', 'lat', 'lon'):
                            if e.time.shape[0] == 5:
                                available.loc[var, name] = 'x'

In [17]:
available.columns = [f"{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}" for exp in experiments.values()]

In [18]:
available

,ssp370-2025-x1,ssp370-2025-x2,ssp370-2025-x3,ssp370-2025-x4,ssp370-2025-x5,piControl-x1,piControl-x2,piControl-x3,piControl-x4,piControl-x5
EFLX_GNET,,,x,x,x,,,x,x,x
EFLX_LH_TOT,,,x,x,x,,,x,x,x
FAREA_BURNED,,,x,x,x,,,x,x,x
FIRE,x,x,,,,x,x,,,
FLDS,,x,x,x,x,,x,x,x,x
FLNS,,x,x,x,x,,x,x,x,x
FSDS,,x,x,x,x,,x,x,x,x
FSH,,,x,x,x,,,x,x,x
FSNO,,,x,x,x,,,x,x,x
FSNS,,x,x,x,x,,x,x,x,x


In [19]:
available.columns = [f"x{exp.experiment_identifier[1]}" for exp in experiments.values()]

In [30]:
print("|                | ssp375-2025                      | piControl                        |")


|                | ssp375-2025                      | piControl                        |


In [31]:
print(available.iloc[:,:5].to_markdown())

|                | x1   | x2   | x3   | x4   | x5   |
|:---------------|:-----|:-----|:-----|:-----|:-----|
| EFLX_GNET      |      |      | x    | x    | x    |
| EFLX_LH_TOT    |      |      | x    | x    | x    |
| FAREA_BURNED   |      |      | x    | x    | x    |
| FIRE           | x    | x    |      |      |      |
| FLDS           |      | x    | x    | x    | x    |
| FLNS           |      | x    | x    | x    | x    |
| FSDS           |      | x    | x    | x    | x    |
| FSH            |      |      | x    | x    | x    |
| FSNO           |      |      | x    | x    | x    |
| FSNS           |      | x    | x    | x    | x    |
| GPP            | x    | x    | x    | x    | x    |
| HR             | x    | x    |      |      |      |
| ICEFRAC        | x    | x    | x    | x    | x    |
| LHFLX          |      | x    | x    | x    | x    |
| NBP            | x    | x    | x    | x    | x    |
| PBOT           |      |      | x    | x    | x    |
| PRECC          | x    | x 